In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import datetime

import warnings
warnings.filterwarnings(action="ignore", category=UserWarning)

In [ ]:
def read_data():
  # Load CSV into dataframe and format
  df = pd.read_csv('final_daily.csv')
  df['date']=pd.to_datetime(df['date'])
  df=df[df['date']<datetime.datetime(2023,1,1)]

  variable = 'demand'

  VAL_PERC = 0.30

  n_train = int(len(df[:-365]) * (1 - VAL_PERC))

  dataframe_train = df[variable][0:-365]
  #train_df = dataframe_train.values.reshape(-1, 1)

  dataframe_test = df[variable][-365:]
  test_df = dataframe_test.values.reshape(-1, 1)

  dataframe_dataset = df[variable][0:df.shape[0]]
  #dataset= dataframe_dataset.values.reshape(-1, 1)

  return df, dataframe_train, dataframe_test, test_df, dataframe_dataset

In [ ]:
#df, train, test, dataset = read_data()
df, train, test, testY, dataset = read_data()

In [ ]:
class KalmanFilter:
    def __init__(self, x, F, P, Q, H, u=None, G=None):
        self.x = x
        self.F = F
        self.P = P
        self.Q = Q
        self.H = H
        self.u = u
        self.G = G

        if G is None or u is None:
            self.u = np.zeros_like(x)
            self.G = np.zeros_like(F)

        self.predict()


    def predict(self):
        self.x = self.F @ self.x + self.G @ self.u
        self.P = self.F @ self.P @ self.F.T + self.Q


    def update(self, z, R):
        K = self.P + self.H.T @ np.linalg.inv(self.H @ self.P @ self.H.T + R)
        self.x = self.x + K @ (z - self.H @ self.x)
        self.P = self.P - K @ self.H @ self.P

In [ ]:
class KalmanFilter1D:
    def __init__(self, x, F, P, Q):
        self.x = x
        self.F = F
        self.P = P
        self.Q = Q

        self.predict()


    def predict(self):
        self.x = self.F * self.x
        self.P = self.P * self.F**2  + self.Q


    def update(self, z, R):
        K = self.P / (self.P + R)
        self.x = self.x + K * (z - self.x)
        self.P = self.P - K * self.P

In [ ]:
x = 0
F = 1
P = 1
Q = 50
zs = test.values.reshape(-1)
Rs = np.ones_like(zs) * 100

kalman = KalmanFilter1D(x, F, P, Q)
corrections = []
predictions = []
verbose = False

for t, (z, R) in enumerate(zip(zs, Rs)):
    predictions.append(kalman.x)
    if verbose: print(f"Time {t+1}: Prediction for current state: {kalman.x:.4f}.")

    kalman.update(z, R)
    corrections.append(kalman.x)
    if verbose: print(f"Time {t+1}: Measurement: {z:.4f}. Correction: {kalman.x:.4f}")

    kalman.predict()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=test.index, y=zs,
                    mode='lines+markers',
                    name='measurements'))
fig.add_trace(go.Scatter(x=test.index, y=predictions,
                    mode='lines+markers',
                    name='predictions'))
#fig.add_trace(go.Scatter(x=df_spain.index, y=corrections,
#                    mode='lines+markers', 
#                    name='after corrections'))

fig.update_layout(hovermode="x",
                  title="Kalman Filter",
                  xaxis_title="Date",
                  yaxis_title="Count")

fig.show()

In [ ]:
np.square(np.array(predictions)[-365:].reshape(-1, 1) - testY).mean()

1108128912.5388489

In [ ]:
np.abs(np.array(predictions)[-365:].reshape(-1, 1) - testY).mean()

19236.023987113305

In [ ]:
def mape(x, y):
  error = abs(x-y)/abs(x)
  error[error > 1] = 1
  return np.mean(error)

mape(np.array(predictions)[-365:].reshape(-1, 1), testY)

<ipython-input-10-98e4fac885b7>:2: RuntimeWarning:

divide by zero encountered in true_divide



0.05357406444054949